In [1]:
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/lingualsense/merged_dataset.csv


In [2]:
df = pd.read_csv('/kaggle/input/lingualsense/merged_dataset.csv')
print(df.head())

                                                Text  Language
0  klement gottwaldi surnukeha palsameeriti ning ...  Estonian
1  sebes joseph pereira thomas  på eng the jesuit...   Swedish
2  ถนนเจริญกรุง อักษรโรมัน thanon charoen krung เ...      Thai
3  விசாகப்பட்டினம் தமிழ்ச்சங்கத்தை இந்துப் பத்திர...     Tamil
4  de spons behoort tot het geslacht haliclona en...     Dutch


In [3]:
df.rename(columns={'Text': 'text', 'Language': 'language'}, inplace=True)

df.dropna(subset=['text', 'language'], inplace=True)

from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df['language_encoded'] = label_encoder.fit_transform(df['language'])

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df['text'], df['language_encoded'], test_size=0.2, random_state=42
)

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf.fit_transform(X_train).toarray()
X_test_tfidf = tfidf.transform(X_test).toarray()

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Embedding, Dropout

# Define the model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_tfidf.shape[1],)),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(len(label_encoder.classes_), activation='softmax')
])


model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 128)                 │         640,128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 30)                  │           1,950 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 650,334 (2.48 MB)

 Trainable params: 650,334 (2.48 MB)

 Non-trainable params: 0 (0.00 B)

In [7]:
model.fit(X_train_tfidf, y_train, validation_data=(X_test_tfidf, y_test), epochs=10, batch_size=32)

Epoch 1/10
809/809 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.6325 - loss: 1.6653 - val_accuracy: 0.9266 - val_loss: 0.2740
Epoch 2/10
809/809 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9325 - loss: 0.2638 - val_accuracy: 0.9298 - val_loss: 0.2503
Epoch 3/10
809/809 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9394 - loss: 0.2161 - val_accuracy: 0.9309 - val_loss: 0.2506
Epoch 4/10
809/809 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9463 - loss: 0.1784 - val_accuracy: 0.9300 - val_loss: 0.2475
Epoch 5/10
809/809 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9499 - loss: 0.1689 - val_accuracy: 0.9258 - val_loss: 0.2519
Epoch 6/10
809/809 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9511 - loss: 0.1589 - val_accuracy: 0.9289 - val_loss: 0.2672
Epoch 7/10
809/809 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9528 - loss: 0.1560 - val_accuracy: 0.9287 - val_loss: 0.2756
Epoch 8/10
809/809 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9529 - loss: 0.1467 - val_accuracy: 0.

In [8]:
import pickle

# Save the model
model.save('language_detection_gru.h5')

# Save the TF-IDF vectorizer
with open('tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(tfidf, f)

# Save the LabelEncoder
with open('label_encoder.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)